In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from collections import Counter
import json
from itertools import islice

In [2]:
with open("Hiztegiak/hiztegi_ez_eus_log.json","r") as file:
    hiztegi_ez_eus = json.load(file)
with open("Hiztegiak/hiztegi_log_eus.json","r") as file:
    hiztegi_eus = json.load(file)

In [67]:
print(len(hiztegi_ez_eus))
print(len(hiztegi_eus))
print(list(hiztegi_ez_eus.values())[-1])
print(list(hiztegi_eus.values())[-1])

4095125
3558797
-8.785736979579099
-8.726195979217117


In [3]:
def testu_garbia(text):
    """ 
    Testu bat jaso eta, edukia aztertzeko orduan kodifikazioarekin arazorik egon ez dadin,
    dena minuskuletan idatzi, puntuazio markak eta zenbakiak ezabatzen ditu. 
    Argumentuak: Edozein eratan idatzitako testu bat.
    Itzultzitakoa: Testu hori bera lista baten antolatua. Listako osagai bakoitza hitz bat izanik
    zenbakiak eta puntuazio markak ezabatuz.
    """
    text = text.lower() #Minuskulara pasatu
    text = re.sub(r'[\W_0-9]+', ' ', text) #Puntuazio markak eta zenbakiak ezabatu
    return [word for word in text.split() if len(word) > 1]

## Balidazioa

In [4]:
def klasifikazio_boolearra(text, hiztegi_eus, hiztegi_ez_eus, oov, theta):
    """ 
    Testu bat jaso eta euskaraz idatzia dagoen edo ez bueltatzen du, testua bi hiztegietara
    zenbat hurbiltzen den kontsideratuz.
    
    Argumentuak:
        - text (str): Testua.
        - hiztegi_eus (dict): Euskarazko hitz-maiztasunen hiztegia.
        - hiztegi_ez_eus (dict): Ez-euskarazko hitz-maiztasunen hiztegia.
    
    Itzulera:
        - bool: True (euskaraz idatzia), False (ez).
    """
    garbitu = testu_garbia(text)
    kop = len(garbitu)  
    maizt_eus = 0  
    maizt_ez_eus = 0  
    for word in garbitu:  
        if word in hiztegi_eus:
            maizt_eus += hiztegi_eus[word]
        else:
            maizt_eus += oov
    for word in garbitu:  
        if word in hiztegi_ez_eus:
            maizt_ez_eus += hiztegi_ez_eus[word]
        else:
            maizt_ez_eus += oov
    if kop != 0:
        balio = (maizt_eus-maizt_ez_eus)/kop
    else:
        balio = -9
    if balio > theta :
        return True
    else:
        return False

In [100]:
testua ="Die Technologie hat unser tägliches Leben stark beeinflusst. Die Erfindung des Internets und der Smartphones hat die Art und Weise, wie wir kommunizieren und Informationen abrufen, verändert."
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")

Euskaraz dago idatzia? Ez


In [6]:
testua ="Teknologiak gure eguneroko bizitzan eragin handia izan du. Internetaren eta telefono adimendunen asmakuntzak gure komunikazio eta informazio sarbidea eraldatu du."
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9)
print(f"Euskara dago idatzia? {'Bai' if badau else 'Ez'}")

Euskara dago idatzia? Bai


In [127]:
testua ="La tecnología ha impactado enormemente nuestras vidas diarias. La invención de internet y los teléfonos inteligentes ha transformado la manera en que nos comunicamos y accedemos a la información."
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")

Euskaraz dago idatzia? Ez


In [5]:
with open("Data/Validation/validation.json","r") as file:
    val_ez_eus = json.load(file)
with open("Data/Validation/validation_eus.jsonl","r") as file:
    val_eus = json.load(file)

theta = np.arange(0.9, 1.1, 0.01)

### Euskera

In [13]:
max_theta = 0
maxim = 1
for value in theta:
    eusk = 0
    bai = []
    eusk_ez = 0
    ez = []
    for text in val_ez_eus:
        badau = klasifikazio_boolearra(text,hiztegi_eus,hiztegi_ez_eus,-9, value)
        if badau:
            eusk += 1
            bai.append(text)
        val_erdera = 100*eusk/len(val_ez_eus)
    for text in val_eus:
        badau = klasifikazio_boolearra(text,hiztegi_eus,hiztegi_ez_eus,-9, value)
        if not badau:
            eusk_ez += 1
            ez.append(text)
    balio = (eusk+eusk_ez)/(len(val_ez_eus)+len(val_eus))
    if balio < maxim:
            maxim = balio
            max_theta = value  
    print(f'%{100*balio}')

print(f'%{100*maxim}')
print(max_theta)

%0.31318374876657085
%0.3046033720880347
%0.2874426187309623
%0.28315243039169424
%0.28315243039169424
%0.2788622420524261
%0.2788622420524261
%0.2788622420524261
%0.2788622420524261
%0.2788622420524261
%0.274572053713158
%0.2702818653738899
%0.2574113003560856
%0.2574113003560856
%0.25312111201681753
%0.25312111201681753
%0.2617014886953537
%0.2617014886953537
%0.26599167703462184
%0.26599167703462184
%0.26599167703462184
%0.25312111201681753
1.04


In [16]:
eusk = 0
bai = []
eusk_ez = 0
ez = []
for text in val_ez_eus:
    badau = klasifikazio_boolearra(text,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
    if badau:
        eusk += 1
        bai.append(text)
    val_erdera = 100*eusk/len(val_ez_eus)
for text in val_eus:
    badau = klasifikazio_boolearra(text,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
    if not badau:
        eusk_ez += 1
        ez.append(text)
print(f'Euskarazko errorea: %{100*eusk/len(val_eus)}')
print(f'Euskarazko esaldi kop:{eusk_ez}')
print(f'Erdarazko errorea: %{100*eusk/len(val_ez_eus)}')
print(f'Erdarazko esaldi kop:{eusk}')

Euskarazko errorea: %0.3026634382566586
Euskarazko esaldi kop:29
Erdarazko errorea: %0.22393073076061804
Erdarazko esaldi kop:30


In [20]:
luz = len(val_ez_eus)+len(val_eus)
err = 29 + 30
errorea = 100*err/luz
print(errorea)
print(100-errorea)

0.25312111201681753
99.74687888798319


# Test

In [22]:
with open("Data/Test/test.json","r") as file:
    ez_eus_test = json.load(file)
with open("Data/Test/test_eus.jsonl","r") as file:
    eus_test = json.load(file)

In [31]:
eusk = 0
bai = []
eusk_ez = 0
ez = []
for text in ez_eus_test:
    badau = klasifikazio_boolearra(text,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
    if badau:
        eusk += 1
        bai.append(text)
    val_erdera = 100*eusk/len(val_ez_eus)
for text in eus_test:
    badau = klasifikazio_boolearra(text,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
    if not badau:
        eusk_ez += 1
        ez.append(text)
print(f'Euskarazko errorea: %{100*eusk/len(eus_test)}')
print(f'Euskarazko esaldi kop:{eusk_ez}')
print(f'Erdarazko errorea: %{100*eusk/len(ez_eus_test)}')
print(f'Erdarazko esaldi kop:{eusk}')

Euskarazko errorea: %0.2523722996163941
Euskarazko esaldi kop:28
Erdarazko errorea: %0.18641413764819925
Erdarazko esaldi kop:25


In [32]:
luz = len(ez_eus_test)+len(eus_test)
err = 28 + 25
errorea = 100*err/luz
print(errorea)
print(100-errorea)

0.2273019685208217
99.77269803147918


In [25]:
testua = "Halare saiatukonaz zoze bilatzen"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Aunkesea urte bukaerarte"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Ni azaruan niju seuraski ya lanea bilboa"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Igual irail bukaeran barnara niju"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Ta porcierto igandin atsaldeko 7tan TOC donostin gratis"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Lautako busa hartukoit"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Mendiya da pero justo egun hortan zehar erromeriyak taola dozte"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Osa 27yan posiblegu mendi buelta gozo bat in ta 28yan artikutza"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Hombre posiblebaut zuekin jun bai"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = "Nor da?"
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1.04)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")

Euskaraz dago idatzia? Bai
Euskaraz dago idatzia? Bai
Euskaraz dago idatzia? Ez
Euskaraz dago idatzia? Bai
Euskaraz dago idatzia? Ez
Euskaraz dago idatzia? Ez
Euskaraz dago idatzia? Bai
Euskaraz dago idatzia? Bai
Euskaraz dago idatzia? Bai
Euskaraz dago idatzia? Ez


In [33]:
testua = 'Pues ez dakit, es que ez dut uste oso sortzailea naizenik ni, sabes, enplan, gero ere sortzerako orduan como que naizela oso, formula asko errepikatzen ditut etab.'
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")
testua = 'A ver da Como debate bat ya filosofikoago, baina sinmas ez dakit osa que más da'
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9, 1)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")

Euskaraz dago idatzia? Bai
Euskaraz dago idatzia? Bai


In [171]:
testua = 'Arront politta den mendi puntan gotti igota auzko aiekaruntz beitu ezkeo karrikara dijun kamiyua ikusiko dek'
badau = klasifikazio_boolearra(testua,hiztegi_eus,hiztegi_ez_eus,-9)
print(f"Euskaraz dago idatzia? {'Bai' if badau else 'Ez'}")

Euskaraz dago idatzia? Bai
